In [18]:
# 📦 STEP 1: Install required packages
!pip install -q streamlit transformers sentencepiece
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


(Reading database ... 133673 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.5.0) over (2025.5.0) ...
Setting up cloudflared (2025.5.0) ...
Processing triggers for man-db (2.10.2-1) ...


In [19]:
%%writefile app.py
import streamlit as st
from transformers import MarianMTModel, MarianTokenizer

st.set_page_config(page_title="🌐 Language Translator", page_icon="🌍")
st.markdown("<h1 style='text-align: center; color: #4CAF50;'>🌍 Universal Language Translator</h1>", unsafe_allow_html=True)

@st.cache_resource
def get_model(src, tgt):
    model_name = f'Helsinki-NLP/opus-mt-{src}-{tgt}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

def translate_text(text, src, tgt):
    try:
        tokenizer, model = get_model(src, tgt)
        inputs = tokenizer(text, return_tensors="pt", padding=True)
        translated = model.generate(**inputs)
        return tokenizer.decode(translated[0], skip_special_tokens=True)
    except:
        return None

languages = {
    "English": "en",
    "French": "fr",
    "German": "de",
    "Hindi": "hi",
    "Gujarati": "gu",
    "Marathi": "mr",
    "Bangla": "bn",
    "Spanish": "es",
    "Japanese": "ja",
    "Russian": "ru",
    "Arabic": "ar",
    "Chinese": "zh"
}

with st.container():
    st.write("### Enter text to translate:")
    text = st.text_area("", height=150)

    col1, col2 = st.columns(2)
    with col1:
        src_lang_name = st.selectbox("🔤 From Language", list(languages.keys()), index=0)
    with col2:
        tgt_lang_name = st.selectbox("🈯 To Language", list(languages.keys()), index=1)

    src = languages[src_lang_name]
    tgt = languages[tgt_lang_name]

    if st.button("🚀 Translate", use_container_width=True):
        if src == tgt:
            st.warning("Please select different source and target languages.")
        else:
            result = translate_text(text, src, tgt)
            if result:
                st.success("✅ Direct Translation:")
                st.markdown(f"<div style='background-color:#e8f5e9;padding:10px;border-radius:5px;'>{result}</div>", unsafe_allow_html=True)
            else:
                st.warning("Direct translation not available. Trying via English...")
                inter = translate_text(text, src, "en")
                if inter:
                    final = translate_text(inter, "en", tgt)
                    if final:
                        st.success("✅ Translated via English:")
                        st.markdown(f"<div style='background-color:#fff3e0;padding:10px;border-radius:5px;'>{final}</div>", unsafe_allow_html=True)
                    else:
                        st.error("❌ English → Target translation failed.")
                else:
                    st.error("❌ Source → English translation failed.")


Overwriting app.py


In [20]:
import subprocess
import time

print("📢 Launching Streamlit app...")
streamlit_proc = subprocess.Popen(["streamlit", "run", "app.py"])

time.sleep(10)

print("🌐 Starting Cloudflare Tunnel...")
tunnel_proc = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

# Print all output lines and manually detect the public URL
public_url = None
while True:
    line = tunnel_proc.stdout.readline()
    if line == "":
        break
    print(line.strip())
    if "trycloudflare.com" in line and "https://" in line:
        parts = line.strip().split()
        for part in parts:
            if "https://" in part and "trycloudflare.com" in part:
                public_url = part
                break
    if public_url:
        print(f"\n✅ Your Public URL: {public_url}")
        break


📢 Launching Streamlit app...
🌐 Starting Cloudflare Tunnel...
2025-06-11T04:48:06Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-06-11T04:48:06Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-06-11T04:48:09Z INF +--------------------------------------------------------------------------------------------+
2025-06-11T04:48:09Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-06-11T04:48:09Z